In [1]:
import opendatasets as od


od.download("https://www.kaggle.com/datasets/mostafaabla/garbage-classification")

Skipping, found downloaded files in "./garbage-classification" (use force=True to force download)


In [2]:
import torch
import sys
import platform
has_gpu = torch.cuda.is_available()
# has_mps = getattr(torch,'has_mps',False)
has_mps = torch.backends.mps.is_built()
device = "mps" if torch.backends.mps.is_built() \
    else "cuda" if torch.cuda.is_available() else "cpu"

print(f"Python Platform: {platform.platform()}")
print(f"PyTorch Version: {torch.__version__}")
print(f"Python {sys.version}")
print("NVIDIA/CUDA GPU is", "available" if has_gpu else "NOT AVAILABLE")
print("MPS (Apple Metal) is", "AVAILABLE" if has_mps else "NOT AzVAILABLE")
print(f"Target device is {device}")

Python Platform: Linux-6.5.9-arch2-1-x86_64-with-glibc2.38
PyTorch Version: 2.1.1+cu121
Python 3.11.0 (main, Mar  1 2023, 18:26:19) [GCC 11.2.0]
NVIDIA/CUDA GPU is available
MPS (Apple Metal) is NOT AzVAILABLE
Target device is cuda


In [3]:
import pathlib
import numpy as np
dataPath = pathlib.Path.cwd() / "garbage-classification"

print(dataPath)

isFile = lambda path : path.is_file()

imagePaths = list(filter(isFile, list(dataPath.rglob("*"))))

pathSamples = np.random.choice(imagePaths, 10000)



/home/coldstuff1/codejam/CodeJam13/model/garbage-classification


In [4]:
from PIL import Image
import torchvision.transforms.v2 as v2

torch.cuda.empty_cache()

transformer = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.uint8, scale=True),
    v2.RandomResizedCrop(size=(256, 256), antialias=True),
    v2.ToDtype(torch.float32, scale=True),  # Normalize expects float input
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

inputImages = []

labelsList = []

rejected = []

for path in pathSamples:
    pathStr = str(path)
    labelNum = -1
    if 'battery' in pathStr:
        labelNum = 0
    elif 'biological' in pathStr:
        labelNum = 1
    elif 'brown-glass' in pathStr:
        labelNum = 2
    elif 'cardboard' in pathStr:
        labelNum = 3
    elif 'clothes' in pathStr:
        labelNum = 4
    elif 'green-glass' in pathStr:
        labelNum = 5
    elif 'metal' in pathStr:
        labelNum = 6
    elif 'paper' in pathStr:
        labelNum = 7
    elif 'plastic' in pathStr:
        labelNum = 8
    elif 'shoes' in pathStr:
        labelNum = 9
    elif 'trash' in pathStr:
        labelNum = 10
    elif 'white-glass' in pathStr:
        labelNum = 11
    if labelNum == -1:
        continue
    img = Image.open(pathStr)
    tensor = transformer(img).to(device)
    if(tensor.shape[0] != 3):
        rejected.append(pathStr)
        continue
    labelsList.append(labelNum)
    inputImages.append(torch.transpose(tensor, 0, 2))
    


dataset = torch.stack(inputImages).to(device)
labels = torch.ByteTensor(labelsList)


print(dataset.shape)
print(labels.shape)



OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacty of 5.62 GiB of which 2.75 MiB is free. Including non-PyTorch memory, this process has 5.61 GiB memory in use. Of the allocated memory 4.15 GiB is allocated by PyTorch, and 1.38 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
train_size = int(0.8* len(dataset))
validate_size = int(0.1* len(dataset))
test_size = int(0.1* len(dataset))

print(train_size)
print(validate_size)
print(test_size)

7988
998
998


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 12)  # 10 classes in Fashion MNIST
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2)
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
model = CNN().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(25):  # Adjust the number of epochs as needed
    running_loss = 0.0
    for i in range(train_size):
        inputs, labels = dataset[i].to(device), labels[i].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / train_size}")

print("Finished Training")


RuntimeError: Input type (unsigned char) and bias type (float) should be the same